In [86]:
# import sklearn.neighbors._base
# import sys
# sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

import datetime
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import make_pipeline
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

In [110]:
test = pd.read_csv("./test_anomaly.csv", delimiter=",", sep='.')
# train = pd.read_csv("./train.csv", delimiter=",", sep='.')
train = pd.read_csv("./train_anomaly.csv", delimiter=",", sep='.')

In [88]:
pd.set_option('display.max_columns', None)
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,anomaly_col
0,1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,Norm
1,2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,Anomaly
2,3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,Anomaly
3,4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,Norm
4,6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,Norm


In [89]:
from catboost import Pool, CatBoostRegressor, cv

In [90]:
train['target'] = 100 * train['target']

In [8]:
# 73.699184       0.532424        73.406661

In [91]:
num_train = train.select_dtypes([int, float])
cat_train = train.select_dtypes(object)

num = list(num_train.drop(['target'],axis=1))
cat = list(cat_train)

In [7]:
# from sklearn.preprocessing import KBinsDiscretizer

# pipeline_num = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaling', StandardScaler()),  
#     ('normal', PowerTransformer()), 
#     ('bins', KBinsDiscretizer())
# ])

# pipeline_cat = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoding', OneHotEncoder(handle_unknown='ignore')),
# ])

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', pipeline_num, num),
        ], remainder="passthrough")

In [78]:
df = preprocessor.fit_transform(train)

In [31]:
import optuna
from sklearn.metrics import mean_squared_error

In [103]:
X = train.drop(["target", "id"], axis=1)
y = train["target"]

# X = df[:, :81].astype('str')
# y = df[:, 81]


train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.7, random_state=42)


# train_pool = Pool(train_x, 
#               train_y, 
#               cat_features=list(range(81)))
# test_pool = Pool(valid_x, 
#               valid_y, 
#               cat_features=list(range(81)))
   
    
train_pool = Pool(train_x, 
              train_y, 
              cat_features=list(range(10)) + [24])
test_pool = Pool(valid_x, 
              valid_y, 
              cat_features=list(range(10)) + [24])

def objective(trial):

    

#     param = {
#         "iterations": 200, 
#         "loss_function": "RMSE",
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
#         "depth": trial.suggest_int("depth", 1, 12),
#         "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
#         "bootstrap_type": trial.suggest_categorical(
#             "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
#         ),
#         "used_ram_limit": "3gb",
#     }

#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    param = {
        'iterations' : trial.suggest_int('iterations', 150, 500),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                                    
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 0.5),
    }
    
    gbm = CatBoostRegressor(loss_function ="RMSE", **param)

    gbm.fit(train_pool, eval_set=test_pool, verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    rmse = mean_squared_error(valid_y, preds, squared = False)
    return rmse

In [104]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, timeout=600)

[I 2021-08-18 22:09:47,069] A new study created in memory with name: no-name-7fcf42db-dfdf-4562-a3ba-8a55a1dfa389
[I 2021-08-18 22:12:58,136] Trial 0 finished with value: 72.6085959501892 and parameters: {'iterations': 425, 'depth': 8, 'random_strength': 36, 'bagging_temperature': 0.22228849444998328, 'learning_rate': 0.13417348761409664}. Best is trial 0 with value: 72.6085959501892.
[I 2021-08-18 22:15:00,872] Trial 1 finished with value: 73.71522862914077 and parameters: {'iterations': 496, 'depth': 7, 'random_strength': 32, 'bagging_temperature': 0.15682138316240113, 'learning_rate': 0.02690458065542879}. Best is trial 0 with value: 72.6085959501892.
[I 2021-08-18 22:16:27,178] Trial 2 finished with value: 72.72198615929992 and parameters: {'iterations': 225, 'depth': 7, 'random_strength': 37, 'bagging_temperature': 2.8514761744022707, 'learning_rate': 0.35356661987813315}. Best is trial 0 with value: 72.6085959501892.
[I 2021-08-18 22:16:38,321] Trial 3 finished with value: 74.529

In [105]:
best_params = study.best_trial.params
best_params

{'iterations': 353,
 'depth': 4,
 'random_strength': 42,
 'bagging_temperature': 0.027785143340172104,
 'learning_rate': 0.4840596931491217}

In [107]:
X = train.drop(["target", "id"], axis=1)
y = train.target

# X = df[:, :81].astype('str')
# y = df[:, 81]

train_pool = Pool(X, 
                  y, 
                  cat_features=list(range(10)) + [24])

params = best_params.copy()
params["loss_function"] ="RMSE"

In [109]:
print(cv(train_pool,
            params,
            fold_count=5, 
            plot=False))

0:	learn: 431.7722339	test: 431.7550585	best: 431.7550585 (0)	total: 3.2s	remaining: 18m 47s
1:	learn: 231.7599604	test: 231.7349884	best: 231.7349884 (1)	total: 5.73s	remaining: 16m 46s
2:	learn: 135.5250449	test: 135.4965968	best: 135.4965968 (2)	total: 8.22s	remaining: 15m 58s
3:	learn: 94.6518414	test: 94.6263056	best: 94.6263056 (3)	total: 10.1s	remaining: 14m 42s
4:	learn: 80.3121667	test: 80.2884307	best: 80.2884307 (4)	total: 12.3s	remaining: 14m 17s
5:	learn: 76.0375540	test: 76.0169044	best: 76.0169044 (5)	total: 15s	remaining: 14m 25s
6:	learn: 74.8400829	test: 74.8262366	best: 74.8262366 (6)	total: 17.1s	remaining: 14m 7s
7:	learn: 74.5221359	test: 74.5094926	best: 74.5094926 (7)	total: 18.7s	remaining: 13m 24s
8:	learn: 74.4303937	test: 74.4194265	best: 74.4194265 (8)	total: 20.7s	remaining: 13m 9s
9:	learn: 74.3733161	test: 74.3638512	best: 74.3638512 (9)	total: 22.5s	remaining: 12m 52s
10:	learn: 74.3659388	test: 74.3579125	best: 74.3579125 (10)	total: 24.5s	remaining: 1

89:	learn: 72.2286105	test: 72.4547769	best: 72.4547769 (89)	total: 3m 45s	remaining: 10m 58s
90:	learn: 72.2200838	test: 72.4494615	best: 72.4494615 (90)	total: 3m 47s	remaining: 10m 56s
91:	learn: 72.2109570	test: 72.4434042	best: 72.4434042 (91)	total: 3m 50s	remaining: 10m 53s
92:	learn: 72.2029820	test: 72.4403089	best: 72.4403089 (92)	total: 3m 53s	remaining: 10m 52s
93:	learn: 72.1912775	test: 72.4311930	best: 72.4311930 (93)	total: 3m 55s	remaining: 10m 50s
94:	learn: 72.1794469	test: 72.4223530	best: 72.4223530 (94)	total: 3m 58s	remaining: 10m 47s
95:	learn: 72.1688349	test: 72.4140178	best: 72.4140178 (95)	total: 4m 1s	remaining: 10m 45s
96:	learn: 72.1623097	test: 72.4112478	best: 72.4112478 (96)	total: 4m 3s	remaining: 10m 43s
97:	learn: 72.1532669	test: 72.4084629	best: 72.4084629 (97)	total: 4m 6s	remaining: 10m 41s
98:	learn: 72.1459380	test: 72.4039784	best: 72.4039784 (98)	total: 4m 9s	remaining: 10m 39s
99:	learn: 72.1352346	test: 72.3959984	best: 72.3959984 (99)	tot

KeyboardInterrupt: 

In [111]:
test_pool = Pool(test.drop(["id"], axis=1), 
                 cat_features=list(range(10)) + [24])

model = CatBoostRegressor(**params)
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)

0:	learn: 74.4514640	total: 332ms	remaining: 1m 56s
1:	learn: 74.4416766	total: 713ms	remaining: 2m 5s
2:	learn: 74.4101600	total: 1.05s	remaining: 2m 2s
3:	learn: 74.3612299	total: 1.35s	remaining: 1m 58s
4:	learn: 74.3188670	total: 1.66s	remaining: 1m 55s
5:	learn: 74.2407649	total: 1.8s	remaining: 1m 44s
6:	learn: 74.2359852	total: 2.07s	remaining: 1m 42s
7:	learn: 74.1987077	total: 2.37s	remaining: 1m 42s
8:	learn: 74.1709004	total: 2.67s	remaining: 1m 42s
9:	learn: 74.1659685	total: 3.05s	remaining: 1m 44s
10:	learn: 74.1557542	total: 3.44s	remaining: 1m 46s
11:	learn: 74.1435253	total: 3.85s	remaining: 1m 49s
12:	learn: 74.0922613	total: 4.17s	remaining: 1m 49s
13:	learn: 74.0595707	total: 4.47s	remaining: 1m 48s
14:	learn: 74.0595707	total: 4.54s	remaining: 1m 42s
15:	learn: 74.0588860	total: 4.72s	remaining: 1m 39s
16:	learn: 74.0588860	total: 4.79s	remaining: 1m 34s
17:	learn: 74.0360313	total: 5.08s	remaining: 1m 34s
18:	learn: 74.0360305	total: 5.21s	remaining: 1m 31s
19:	le

155:	learn: 71.7631959	total: 52.4s	remaining: 1m 6s
156:	learn: 71.7551467	total: 52.8s	remaining: 1m 5s
157:	learn: 71.7535960	total: 53.1s	remaining: 1m 5s
158:	learn: 71.7440712	total: 53.4s	remaining: 1m 5s
159:	learn: 71.7426241	total: 53.8s	remaining: 1m 4s
160:	learn: 71.7383523	total: 54.2s	remaining: 1m 4s
161:	learn: 71.7378562	total: 54.4s	remaining: 1m 4s
162:	learn: 71.7332609	total: 54.8s	remaining: 1m 3s
163:	learn: 71.7291948	total: 55.2s	remaining: 1m 3s
164:	learn: 71.7217277	total: 55.5s	remaining: 1m 3s
165:	learn: 71.7193912	total: 55.9s	remaining: 1m 2s
166:	learn: 71.7155568	total: 56.2s	remaining: 1m 2s
167:	learn: 71.7117349	total: 56.6s	remaining: 1m 2s
168:	learn: 71.7039060	total: 56.9s	remaining: 1m 1s
169:	learn: 71.6991943	total: 57.4s	remaining: 1m 1s
170:	learn: 71.6952929	total: 57.7s	remaining: 1m 1s
171:	learn: 71.6895215	total: 58s	remaining: 1m 1s
172:	learn: 71.6862540	total: 58.4s	remaining: 1m
173:	learn: 71.6826381	total: 58.7s	remaining: 1m
1

309:	learn: 71.2436487	total: 1m 50s	remaining: 15.3s
310:	learn: 71.2410837	total: 1m 51s	remaining: 15s
311:	learn: 71.2372661	total: 1m 51s	remaining: 14.6s
312:	learn: 71.2347885	total: 1m 51s	remaining: 14.3s
313:	learn: 71.2317035	total: 1m 52s	remaining: 13.9s
314:	learn: 71.2286404	total: 1m 52s	remaining: 13.6s
315:	learn: 71.2239756	total: 1m 52s	remaining: 13.2s
316:	learn: 71.2211650	total: 1m 53s	remaining: 12.9s
317:	learn: 71.2186969	total: 1m 53s	remaining: 12.5s
318:	learn: 71.2151913	total: 1m 53s	remaining: 12.1s
319:	learn: 71.2143531	total: 1m 54s	remaining: 11.8s
320:	learn: 71.2129551	total: 1m 54s	remaining: 11.4s
321:	learn: 71.2096009	total: 1m 55s	remaining: 11.1s
322:	learn: 71.2074240	total: 1m 55s	remaining: 10.7s
323:	learn: 71.2041697	total: 1m 55s	remaining: 10.4s
324:	learn: 71.2011995	total: 1m 56s	remaining: 10s
325:	learn: 71.1975813	total: 1m 56s	remaining: 9.65s
326:	learn: 71.1942232	total: 1m 56s	remaining: 9.29s
327:	learn: 71.1910273	total: 1m

In [112]:
test['target'] = preds/100

In [113]:
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,anomaly_col,target
0,0,B,B,B,C,B,B,A,E,E,I,0.296227,0.686757,0.587731,0.392753,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702,0,8.007290
1,5,A,B,A,C,B,C,A,E,C,H,0.543707,0.364761,0.452967,0.929645,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940,0,8.321461
2,15,B,A,A,A,B,B,A,E,D,K,0.408961,0.296129,0.690999,0.740027,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099,0,8.361042
3,16,B,B,A,C,B,D,A,E,A,N,1.031239,0.356062,0.303651,0.895591,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372,0,8.530366
4,17,B,B,A,C,B,C,A,E,C,F,0.530447,0.729004,0.281723,0.444698,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412,0,8.136185


In [114]:
test[['id', 'target']].to_csv('catboost_anomaly.csv', index=False)